In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Cbc")
Pkg.add("PyPlot")
Pkg.add("Images")
Pkg.add("FileIO")
Pkg.add("Statistics")

    Updating registry at `C:\Users\mampane\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\mampane\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\mampane\.julia\environments\v1.11\Manifest.toml`
Precompiling project...
   1630.7 ms  ✓ HistogramThresholding
   2545.0 ms  ✓ ImageTransformations
   2694.2 ms  ✓ ImageShow
   2531.1 ms  ✓ ImageDistances
   2734.2 ms  ✓ Netpbm
   3826.9 ms  ✓ PNGFiles
   3649.6 ms  ✓ Sixel
   3897.0 ms  ✓ JpegTurbo
   2432.3 ms  ✓ ImageBinarization
   2541.5 ms  ✓ ImageContrastAdjustment
   7565.2 ms  ✓ ImageCorners
   8674.0 ms  ✓ ImageQualityIndexes
   4416.0 ms  ✓ Images
  13 dependencies successfully precompiled in 21 seconds. 247 already precompiled.
   Resolving package versions...
  No Changes to `C:\Users\mampane\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\mampane\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\mampa

In [3]:
using JuMP, Cbc, PyPlot, Images, FileIO, Statistics

In [4]:
# input: array with all the given dots (each number represents the colours) and number of colours
# output: solved flows problem
function find_answer(array)
    
    m = Model(Cbc.Optimizer) 
    optimize!(m)
    xsize, ysize = size(array)
    n = Int8(findmax(array)[1])
    
    # variable to control the colours of each blocks (and constraints)
    @variable(m, ans[0:xsize+1,0:ysize+1,1:n], Bin)
    @constraint(m, [i=0:xsize+1,k=1:n], ans[i,0,k] == 0)
    @constraint(m, [i=0:xsize+1,k=1:n], ans[i,ysize+1,k] == 0)
    @constraint(m, [j=0:ysize+1,k=1:n], ans[0,j,k] == 0)
    @constraint(m, [j=0:ysize+1,k=1:n], ans[xsize+1,j,k] == 0)
    
    # only one colour in each square
    @constraint(m, [i=1:xsize, j=1:ysize], sum(ans[i,j,:]) == 1)
    
    # variable for edges between each nodes (one variable for edge between two points and which colour)
    @variable(m, hedges[0:xsize,0:ysize+1,1:n], Bin)
    @variable(m, vedges[0:xsize+1,0:ysize,1:n], Bin)
    @constraint(m, [j=0:ysize+1,k=1:n], hedges[0,j,k] == 0)
    @constraint(m, [j=0:ysize+1,k=1:n], hedges[xsize,j,k] == 0)
    @constraint(m, [i=0:xsize+1,k=1:n], vedges[i,0,k] == 0)
    @constraint(m, [i=0:xsize+1,k=1:n], vedges[i,ysize,k] == 0)
    
    # each edges can only be there if colours are the same
    @constraint(m, [i=0:xsize,j=0:ysize+1,k=1:n], ans[i,j,k] + ans[i+1,j,k] >= 2*hedges[i,j,k])
    @constraint(m, [i=0:xsize+1,j=0:ysize,k=1:n], ans[i,j,k] + ans[i,j+1,k] >= 2*vedges[i,j,k])
    
    # at most one of the edges is filled for each colour
    @constraint(m, [i=1:xsize-1,j=1:ysize], sum(hedges[i,j,:]) <= 1)
    @constraint(m, [i=1:xsize,j=1:ysize-1], sum(vedges[i,j,:]) <= 1)
    
    # fix the given points in place
    # and fix the number of edges through each of the points
    for i in 1:xsize
        for j in 1:ysize
            if array[i,j] != 0
                co = array[i,j]
                @constraint(m, ans[i,j,co] == 1)
                @constraint(m, hedges[i-1,j,co] + hedges[i,j,co] + vedges[i,j-1,co] + vedges[i,j,co] == 1)
            else
                @constraint(m, [k=1:n], hedges[i-1,j,k]+hedges[i,j,k]+vedges[i,j-1,k]+vedges[i,j,k] == 2*ans[i,j,k])
            end
        end
    end
    
    @objective(m, Max, sum(ans))
    optimize!(m)
    
    ans_array = JuMP.value.(ans)
    hedges_ans = JuMP.value.(hedges)
    vedges_ans = JuMP.value.(vedges)
    final_answer = zeros(Int8, xsize, ysize)
    for x in 1:xsize
        for y in 1:ysize
            for c in 1:n
                if ans_array[x,y,c] == 1
                    final_answer[x,y] = c
                end
            end
        end
    end
    
    return final_answer, hedges_ans, vedges_ans
    
end

p1 = [
    1 0 0 0 1;
    2 0 0 0 3;
    0 0 4 0 4;
    0 0 0 0 0;
    2 5 0 5 3;
    6 0 0 0 6
    ]
p2 = [
    0 1 0 0 2;
    0 0 0 0 4;
    0 0 4 0 3;
    0 0 3 0 0;
    0 0 0 0 0;
    0 0 0 1 2
    ]

@time board_ans, h_ans, v_ans = find_answer(p1)

board_ans

Optimal - objective value 0
Optimal objective 0 - 0 iterations time 0.002
  8.138759 seconds (22.23 M allocations: 1.083 GiB, 4.81% gc time, 98.22% compilation time: <1% of which was recompilation)
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 30 - 0.01 seconds
Cgl0002I 336 variables fixed
Cgl0003I 1 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -30 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rou

6×5 Matrix{Int8}:
 1  1  1  1  1
 2  3  3  3  3
 2  3  4  4  4
 2  3  3  3  3
 2  5  5  5  3
 6  6  6  6  6

In [27]:
h_ans

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 0:6
    Dimension 2, 0:6
    Dimension 3, Base.OneTo(6)
And data, a 7×7×6 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.

In [28]:
v_ans

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 0:7
    Dimension 2, 0:5
    Dimension 3, Base.OneTo(6)
And data, a 8×6×6 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.